In [1]:
%load_ext autoreload
%autoreload 2

# from glob import glob
# import json
# import pickle
from os import path
import itertools
import json

import numpy as np
import pandas as pd
import scipy as sp
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from tqdm.auto import tqdm
# import pandarallel
from IPython.utils.capture import capture_output
with capture_output():
    tqdm.pandas()
#     pandarallel.pandarallel.initialize(progress_bar=True)

import elephant
from neo.core import AnalogSignal
import quantities as pq

from tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.data_analysis.Utilities.utilities import get_stim_events, find_nearest_ind

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False
%matplotlib widget

In [2]:
aois = ['ORBvl', 'ACAd', 'MOs', 'ACAv', 'ILA', 'ORBm', 'PL', 'ORBl', 'VISp', 'VL',
        'RSPv', 'RSPagl', 'MOp', 'RSPd', 'CLA', 'PFC', 'SSp-tr', 'SSp-ll', 'VISrl',# 'CA',
        'LD', 'PO', 'VPM', 'LGd-sh', 'MGv', 'MGm', 'TH', 'ZI', 'VPL', 'RT', 'SSp-bfd']
lois = ['2', '3', '1', '2/3', '4', '5', '6a', '6b']
alois = aois + sum([[x+l for x in aois] for l in lois], [])
area_cmaps = {
    'ACAd':cm.Reds, 'ACAv':cm.Oranges, 'RSPd':cm.Blues, 'RSPv':cm.Purples, 'RSPagl':cm.PuBu, 'ILA':cm.Greens, 'CLA':cm.RdPu,
    'MOs':cm.Greens, 'MOp':cm.copper, 'ORBvl':cm.Reds, 'ORBm':cm.Purples, 'ORBl':cm.RdPu, 'PL':cm.Blues, 'PFC':cm.YlOrBr,
    'SSp-tr':cm.Reds, 'SSp-ll':cm.Blues, 'CA':cm.YlGn, 'LD':cm.Greens, 'PO':cm.Purples, 'VPM':cm.Oranges, 'VISp':cm.Greens, 'LGd-sh':cm.Blues
}
aloi_colors = {f'{a}{l}':area_cmaps.get(a, cm.Greys)(lois.index(l)/len(lois)) for a in aois for l in lois}
aloi_colors.update({a:area_cmaps.get(a, cm.Greys)(0.999) for a in aois})

In [3]:
def get_stim_events_multiple(stim_log, stim_types=None, stim_params=None, sweeps=None):
    if stim_types is None:
        stim_types = stim_log.stim_type.unique()
    if stim_params is None:
        stim_params = stim_log[stim_log.stim_type.isin(stim_types)].parameter.unique()
    if sweeps is None:
        sweeps = stim_log.sweep.unique()
    return stim_log.onset[
        stim_log.stim_type.isin(stim_types)
        & stim_log.parameter.isin(stim_params)
        & stim_log.sweep.isin(sweeps)
    ].rename('event_times')

def separate_running_sweeps(exp, sweeps=None, stim_amps=None, limit=100):
    stim_log = pd.read_csv(exp.stimulus_log_file)
    stim_log.rename_axis(index='stim_id', inplace=True)

    event_times = get_stim_events_multiple(stim_log, ['biphasic'], stim_amps, sweeps)

    running_speed = exp.load_running('pd')

    events = pd.Series(index=event_times, data=range(len(event_times)), name='event_id')
    events = events.reindex(running_speed.index, method='nearest', limit=limit).fillna(-1)
    running_speed = running_speed.to_frame().join(events)

    running_speed_by_event = running_speed.groupby('event_id').mean()[1:]
#     display(running_speed_by_event)

#     display(stim_log.groupby(['parameter', 'sweep']).size())
    stim_log.loc[
        running_speed_by_event[running_speed_by_event>0].dropna().index,
        'sweep'
    ] = stim_log.loc[
        running_speed_by_event[running_speed_by_event>0].dropna().index,
        'sweep'
    ] + 10
    display(stim_log.groupby(['parameter', 'sweep']).size())
    return stim_log

In [4]:
# accessing the Google sheet with experiment metadata in python
# setting up the permissions:
# 1. install gspread (pip install gspread / conda install gspread)
# 2. copy the service_account.json file to '~/.config/gspread/service_account.json'
# 3. run the following:
import gspread
_gc = gspread.service_account() # need a key file to access the account (step 2)
_sh = _gc.open('Zap_Zip-log_exp') # open the spreadsheet
_df = pd.DataFrame(_sh.sheet1.get()) # load the first worksheet
gmetadata = _df.T.set_index(0).T # put it in a nicely formatted dataframe

In [5]:
good_expt = gmetadata[(gmetadata['Units Sorted (X)'].isin(['X']))&(gmetadata['Brain areas assignment'].isin(['X']))&(gmetadata['Npx'].apply(lambda x: len(x.split(','))>2 if x is not None else False))]
good_expt

,mouse_name,exp_name,brain states,stimulation,visual_stim,audio_stim,ISI (sec),stimulus duration (msec),Current (uA),Cortical Area stimulation,N trials per stimulus,EEG bad_channels,Npx,Units Sorted (X),Brain slices (X),Pupil tracking pre-processing,Brain areas assignment,"CCF coordinates stim electrode (surface,tip)","CCF area stim electrode (surface,tip)",Notes
25,mouse551400,estim_vis_2021-01-22_11-07-12,awake/ISO/recovery/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,30/50/70,M2,120/100,"0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18...","F,B,D",X,X,,X,"[ 390,121,439], [ 419,267,442]","MOs1, MOs6a",NO visual stim during the 2nd Recovery. The EE...
28,mouse569062,estim_vis_2021-02-18_11-17-51,awake/ISO/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,15/35/60,M2,120,"6,13","F,B,D",X,X,,X,"[370,146,387], [ 428,300,434]","MOs1, ccg",ISO kept ~1%. White circles for the vis stim. ...
29,mouse569068,estim_vis_2021-03-04_10-51-38,awake/ISO/recovery/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,10/20/40,M2,120,,"F,B,D",X,X,,X,"[383,136,400], [446,289,442]","MOs1, ccg",ISO kept ~1%. White circles for the vis stim. ...
30,mouse569069,estim_vis1_2021-03-11_11-02-08,awake/awake/awake/awake,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,50/60/80,M2,120,"7,8,9,10,11,12,13,14","F,B,C",X,X,,X,"[293,156,456], [331,266,505]","MOs1,PL5",control exp in awake. Something happen to the ...
31,mouse569069,estim_vis2_2021-03-12_10-52-44,awake/ISO/recovery/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,20/40/70,M2,120,,"F,B,C",X,X,,X,"[293,156,456], [331,266,505]","MOs1,PL5",ISO kept ~1%.
32,mouse571619,estim1_2021-03-18_11-41-26,awake/ISO,electrical,N/A,N/A,[3.5 4.5],0.2,20/30/40,SS-cortex,120,"2,3,4","F,B,C",X,X,,X,"[642,45,404], [649,238,431]","SSp-tr1, CA3",all EEG channels very noisy. ISO kept ~1%.
33,mouse571619,estim2_2021-03-19_10-09-01,awake500/awake1200/ISO1200/ISO500,electrical,N/A,N/A,[3.5 4.5],0.2,20/40/60,M2-frontal (2 depths),120,"2,3,4,5,13","F,B,C",X,X,,X,"[273,172,431],[286,314,462]","MOs1, ORBvl6a",2 different depths for the stim electrode in b...
36,mouse569064,estim_vis_2021-04-08_10-28-24,awake/ISO/recovery/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,15/40/60,M2,120,,"F,B,C",X,X,,X,"[369,134,419], [415,272,461]","MOs1, cing",ISO kept ~1%. White circles for the vis stim. ...
37,mouse569073,estim_vis_2021-04-15_10-27-22,awake/ISO/recovery/recovery,electrical/sensory,white,N/A,[3.5 4.5],0.2/250,20/40/70,M2,120,"3,4,13","F,B,C",X,X,,X,"[430,113,421],[437,258,440]","MOp1,MOs6a",ISO kept ~1%. White circles for the vis stim. ...
38,mouse569073,estim_2021-04-16_10-42-44,awake/ISO,electrical,N/A,N/A,[3.5 4.5],0.2,60/80/100,SS-cortex,120,"13,11,10,9,8,7,6,5,4,3,1,0","F,B,C",X,X,,X,not visible,not visible,Control Exp. ISO kept ~1%. White circles for t...


---
phase shifting lfp signals to fix sampling artifact

In [6]:
mouse = 'mouse569062'
expt = 'estim_vis_2021-02-18_11-17-51'
probe = 'probeF'

rec_folder = f'../tiny-blue-dot/zap-n-zip/EEG_exp/{mouse}/{expt}/experiment1/recording1/'
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
samp_rate = exp.ephys_params[probe]['lfp_sample_rate']

lfp = np.memmap(exp.ephys_params[probe]['lfp_continuous'], dtype='int16', mode='r')
lfp = np.reshape(lfp, (int(lfp.size/exp.ephys_params[probe]['num_chs']), exp.ephys_params[probe]['num_chs']))

timestamps = np.load(exp.ephys_params[probe]['lfp_timestamps'])

Experiment type: electrical and sensory stimulation


In [7]:
def freduce(x, axis=None):
    """
    Reduces a spectrum to positive frequencies only
    Works on the last dimension (contiguous in c-stored array)
    :param x: numpy.ndarray
    :param axis: axis along which to perform reduction (last axis by default)
    :return: numpy.ndarray
    """
    if axis is None:
        axis = x.ndim - 1
    siz = list(x.shape)
    siz[axis] = int(np.floor(siz[axis] / 2 + 1))
    return np.take(x, np.arange(0, siz[axis]), axis=axis)


def fexpand(x, ns=1, axis=None):
    """
    Reconstructs full spectrum from positive frequencies
    Works on the last dimension (contiguous in c-stored array)
    :param x: numpy.ndarray
    :param axis: axis along which to perform reduction (last axis by default)
    :return: numpy.ndarray
    """
    if axis is None:
        axis = x.ndim - 1
    # dec = int(ns % 2) * 2 - 1
    # xcomp = np.conj(np.flip(x[..., 1:x.shape[-1] + dec], axis=axis))
    ilast = int((ns + (ns % 2)) / 2)
    xcomp = np.conj(np.flip(np.take(x, np.arange(1, ilast), axis=axis), axis=axis))
    return np.concatenate((x, xcomp), axis=axis)

def fshift(w, s, axis=-1):
    """
    Shifts a 1D or 2D signal in frequency domain, to allow for accurate non-integer shifts
    :param w: input signal
    :param s: shift in samples, positive shifts forward
    :param axis: axis along which to shift (last axis by default)
    :return: w
    """
    # create a vector that contains a 1 sample shift on the axis
    ns = np.array(w.shape) * 0 + 1
    ns[axis] = w.shape[axis]
    dephas = np.zeros(ns)
    np.put(dephas, 1, 1)
    # fft the data along the axis and the dephas
    W = freduce(sp.fft.fft(w, axis=axis), axis=axis)
    dephas = freduce(sp.fft.fft(dephas, axis=axis), axis=axis)
    # if multiple shifts, broadcast along the other dimensions, otherwise keep a single vector
    if not np.isscalar(s):
        s_shape = np.array(w.shape)
        s_shape[axis] = 1
        s = s.reshape(s_shape)
    # apply the shift (s) to the fft angle to get the phase shift
    dephas = np.exp(1j * np.angle(dephas) * s)
    # apply phase shift by broadcasting
    return np.array(np.real(sp.fft.ifft(fexpand(W * dephas, ns[axis], axis=axis), axis=axis)), dtype=w.dtype)

# def get_nsampleshift(n_adc=32):
#     n_channels = 384
#     n_ch_per_adc = n_channels // n_adc
#     return np.array(list(np.round(np.arange(n_ch_per_adc*2)/2-0.01)-n_ch_per_adc/2+0.5)*(n_adc//2))

def apply_sample_time_correction(lfp, n_adc=32, n_channels=384, ofile=None):
    n_ch_per_adc = n_channels // n_adc
    lfp_corrected = np.zeros(lfp.shape, dtype=lfp.dtype)
#     if ofile is not None:
#         lfp_corrected = np.memmap(ofile, dtype=lfp.dtype, mode='w+', shape=lfp.shape)
#     else:
#         lfp_corrected = np.zeros(lfp.shape, dtype=lfp.dtype)

    n_samples_shifted = (np.arange(n_ch_per_adc*2)/2-n_ch_per_adc/2+0.5)[::2] / n_ch_per_adc
    for adc_ch in tqdm(range(0, n_ch_per_adc*2, 2)):
        npx_ch = np.array(sorted(np.r_[np.arange(16)*2*n_ch_per_adc+adc_ch, np.arange(16)*2*n_ch_per_adc+adc_ch+1]))
        lfp_corrected[:, npx_ch] = fshift(lfp[:, npx_ch], n_samples_shifted[adc_ch//2], axis=0)
    if ofile is not None:
        _lfp_corrected = np.memmap(ofile, dtype=lfp.dtype, mode='w+', shape=lfp.shape)
        _lfp_corrected[:] = lfp_corrected[:]
    return lfp_corrected

In [8]:
# test the time shift correction algorithm
mouse = 'mouse569062'
expt = 'estim_vis_2021-02-18_11-17-51'
probe = 'probeF'

rec_folder = f'../tiny-blue-dot/zap-n-zip/EEG_exp/{mouse}/{expt}/experiment1/recording1/'
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
samp_rate = exp.ephys_params[probe]['lfp_sample_rate']

lfp = np.memmap(exp.ephys_params[probe]['lfp_continuous'], dtype='int16', mode='r')
lfp = np.reshape(lfp, (int(lfp.size/exp.ephys_params[probe]['num_chs']), exp.ephys_params[probe]['num_chs']))

try:
    lfp_tx = np.memmap(f"{exp.ephys_params[probe]['lfp_continuous'][:-4]}_corrected.npy", dtype='int16', mode='r')
    lfp_tx = np.reshape(lfp_tx, (int(lfp_tx.size/exp.ephys_params[probe]['num_chs']), exp.ephys_params[probe]['num_chs']))
    lfp_tx = lfp_tx[:20000]
    _lfp_tx = (lfp_tx-lfp_tx.mean(axis=0))/lfp_tx.std(axis=0)
    _lfp_tx = pd.DataFrame(_lfp_tx)
except:
    _lfp_tx = None
    
lfp = lfp[:20000]
_lfp = (lfp-lfp.mean(axis=0))/lfp.std(axis=0)
_lfp = pd.DataFrame(_lfp)

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 3.5), tight_layout=True, sharex=True, sharey=True)

ax1.plot((_lfp - _lfp.rolling(50, center=True).mean())[22], label='ch22')
ax1.plot((_lfp - _lfp.rolling(50, center=True).mean())[24], label='ch24')
ax1.legend(fontsize=9, loc=1)
ax1.set_title('raw LFP (z-scored)')

if _lfp_tx is not None:
    ax2.plot((_lfp_tx - _lfp_tx.rolling(50, center=True).mean())[22], label='ch22')
    ax2.plot((_lfp_tx - _lfp_tx.rolling(50, center=True).mean())[24], label='ch24')
    ax2.legend(fontsize=9, loc=1)
ax2.set_title('corrected LFP (z-scored)')
ax2.set_xlim(5450, 5530)

ax2.set_xlabel(f'time (1/{samp_rate}s units)')

# compute correlation between channel pairs for the following temporal delays
xr = np.arange(-8, 9)

_lfp = (_lfp - _lfp.rolling(50, center=True).mean())
if _lfp_tx is not None:
    _lfp_tx = (_lfp_tx - _lfp_tx.rolling(50, center=True).mean())

# compute for all edge channels (ie channels that are physically consecutive but sampled first and last for maximum difference)
ac = [[_lfp[cpi*24].corr(_lfp[cpi*24-2].shift(x)) for x in xr] for cpi in range(1, 16)]+[[_lfp[cpi*24+1].corr(_lfp[cpi*24-1].shift(x)) for x in xr] for cpi in range(1, 16)]
ac = np.array(ac)
if _lfp_tx is not None:
    ac_tx = [[_lfp_tx[cpi*24].corr(_lfp_tx[cpi*24-2].shift(x)) for x in xr] for cpi in range(1, 16)]+[[_lfp_tx[cpi*24+1].corr(_lfp_tx[cpi*24-1].shift(x)) for x in xr] for cpi in range(1, 16)]
    ac_tx = np.array(ac_tx)

f, ax = plt.subplots(figsize=(3, 2.4), tight_layout=True)
ax.errorbar(xr, ac.mean(0), ac.std(0)/np.sqrt(len(xr)), label='raw', c='k', lw=1, ls='--')
if _lfp_tx is not None:
    ax.errorbar(xr, ac_tx.mean(0), ac_tx.std(0)/np.sqrt(len(xr)), label='corrected', c='k', lw=1)
ax.axvline(0, c='k', lw=0.4)
ax.set_xlabel('time delay')
ax.set_ylabel('correlation')
ax.legend(fontsize=8, loc=3);

Experiment type: electrical and sensory stimulation


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# compute CSD

In [9]:
def load_lfp(rec_folder, probe, repeat=False, separate_running=False):
    exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
    
    if separate_running:
        print('separating running sweeps')
        stim_log = separate_running_sweeps(exp)
    else:
        stim_log = pd.read_csv(exp.stimulus_log_file)
        stim_log.rename_axis(index='stim_id', inplace=True)
    
    # put lfp into a dataframe and add stimulation metadata
    fname = f'../tiny-blue-dot/zap-n-zip/sg/lfp_stim_aligned/{exp.mouse}_{path.basename(path.dirname(exp.experiment_folder))}_{probe}{"_running_separated" if separate_running else ""}.pkl'
    if path.exists(fname) and not repeat:
        lfp = pd.read_pickle(fname)
    else:
        print('Loading and preprocessing LFP.')
        # load lfp
        
        corrected_fname = f"{exp.ephys_params[probe]['lfp_continuous'][:-4]}_corrected.npy"
        if path.exists(corrected_fname):
            lfp = np.memmap(corrected_fname, mode='r+', dtype='int16')
            lfp = np.reshape(lfp, (int(lfp.size/exp.ephys_params[probe]['num_chs']), exp.ephys_params[probe]['num_chs']))
        else:
            print('Applying sampling time correction to LFP. This will take A WHILE!')
            lfp_raw = np.memmap(exp.ephys_params[probe]['lfp_continuous'], dtype='int16', mode='r')
            lfp_raw = np.reshape(lfp_raw, (int(lfp_raw.size/exp.ephys_params[probe]['num_chs']), exp.ephys_params[probe]['num_chs']))
            lfp = apply_sample_time_correction(lfp_raw, ofile=corrected_fname)
        
        samp_rate = exp.ephys_params[probe]['lfp_sample_rate']
        timestamps = np.load(exp.ephys_params[probe]['lfp_timestamps'])

        lfp = pd.DataFrame(lfp, index=timestamps)

        # assign stimulus information at each timestamp, including -1s onwards relative to stimulus to that block
        idx = stim_log.reset_index().set_index('onset')
        idx.index = idx.index - 1
        idx = idx.reindex(timestamps, method='ffill', limit=2500*4).reset_index()

        def _reset_index_time(df):
            df['onset'] = (df.onset - df.onset.iloc[0] - 1).round(4)
            return df
        idx = idx.groupby('stim_id').apply(_reset_index_time).drop(['offset', 'duration'], axis=1).rename(columns={'onset':'time'})

        lfp = lfp.loc[timestamps[idx.index]]
        lfp.index = pd.MultiIndex.from_frame(idx)
        lfp.columns = pd.MultiIndex.from_arrays([
            range(len(lfp.columns)),
            lfp.columns.map(lambda x: -3840+(x+1)//2*20)
        ], names=['channel', 'depth'])

        _t = (idx.sweep+1).fillna(False).astype(bool)
        _t.index = lfp.index
        lfp = lfp[_t]
        lfp = lfp.sort_index()
        
        # remove offsets
        lfp = pd.concat({x:lfp[x] - lfp[x].loc[:-0.001].mean().astype(int) for x in lfp.columns}, axis=1, names=['channel', 'depth'])

        lfp.to_pickle(fname)
    return lfp

def get_mean_lfp(rec_folder, probe, repeat=False, separate_running=False):
    exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
    
    if separate_running:
#         print('separating running sweeps')
        stim_log = separate_running_sweeps(exp)
    else:
        stim_log = pd.read_csv(exp.stimulus_log_file)
        stim_log.rename_axis(index='stim_id', inplace=True)
    
    aparams = list(stim_log.apply(lambda row: (row.stim_type, row.parameter), axis=1).unique())
    lfp = load_lfp(rec_folder, probe, repeat, separate_running=separate_running)
    _lfp = {}
    for p in aparams:
        for sweep in lfp.loc[(slice(None), slice(None), p[0], p[1]), :].index.remove_unused_levels().get_level_values('sweep').unique():
            _lfp[((*p, sweep))] = lfp.loc[(slice(None), slice(None), p[0], p[1], sweep), :].groupby('time').mean()
    _lfp = pd.concat(_lfp, names=['stim_type', 'parameter', 'sweep', 'time'])
    return(_lfp)

In [10]:
def bin_spikes(spikes, bin_size_ms=1, t_start=0, t_end=1e5):
    N = int((t_end-t_start)*1000/bin_size_ms)
    spikes = spikes[(spikes>t_start)&(spikes<t_end)]
    _binned_spikes = pd.Series(
        index=(N * (spikes - t_start) / (t_end - t_start)).astype(int), data=1
    )
    counts = _binned_spikes.reset_index().groupby('index').size()
    times = np.linspace(t_start, t_end, N, endpoint=False)
    binned_spikes = np.zeros(N)
    binned_spikes[counts.index] = counts
    return pd.Series(binned_spikes, index=times, dtype=bool)

def load_spikes(rec_folder, probe, repeat=False):
    exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
    stim_log = pd.read_csv(exp.stimulus_log_file)
    stim_log.rename_axis(index='stim_id', inplace=True)
    
    fname = f'../tiny-blue-dot/zap-n-zip/sg/spikes_stim_aligned/{exp.mouse}_{path.basename(path.dirname(exp.experiment_folder))}_{probe}.pkl'
    if path.exists(fname) and not repeat:
        spikes = pd.read_pickle(fname)
    else:
        print('Loading and preprocessing spiking data.')
        # read raw spike time data
        _spike_times = np.load(exp.ephys_params[probe]['spike_times'], mmap_mode='r')
        _spike_clusters = np.load(exp.ephys_params[probe]['spike_clusters'], mmap_mode='r')
        cluster_metrics = pd.read_csv(exp.ephys_params[probe]['cluster_metrics'], index_col=1).drop('Unnamed: 0', axis=1, errors='ignore')
        cluster_groups = pd.read_csv(exp.ephys_params[probe]['cluster_group'], sep='\t', index_col=0)

        # rearrange into spike times for each cluster
        spike_df = pd.DataFrame(index=_spike_clusters, data=_spike_times, columns=['time'])
        spike_times = spike_df.groupby(level=0).apply(lambda g: g.values[:, 0])

        # keep only good clusters and drop 'noise'
        cluster_metrics = cluster_metrics[cluster_groups.group.isin(['good'])]
        cluster_metrics = cluster_metrics[(cluster_metrics.isi_viol<0.5)&(cluster_metrics.amplitude_cutoff<0.1)]
        spike_times = spike_times.loc[cluster_metrics.index]

        # bin spikes into 1ms bins
        spikes = {}
        t_start = spike_times.apply(lambda x: x.min()).min().round(3)
        t_end = spike_times.apply(lambda x: x.max()).max().round(3)
        for u, t in tqdm(spike_times.items(), total=len(spike_times)):
            spikes[u] = bin_spikes(t, t_start=t_start, t_end=t_end)
        spikes = pd.concat(spikes, axis=1, names='units').rename_axis('time')

        # keep only those spikes that are in a window of interest around stimulus times
        idx = stim_log.reset_index().set_index('onset').rename_axis('time')
        idx.index = idx.index - 1
        idx = idx.reindex(spikes.index, method='ffill', limit=1000*4).reset_index().dropna()
        spikes = spikes.loc[idx.time]
        
        def _reset_index_time(df):
            df['time'] = (df.time - df.time.iloc[0] - 1).round(3)
            return df
        idx = idx.groupby('stim_id').apply(_reset_index_time).drop(['offset', 'duration'], axis=1)
        
        # set index and columns with useful information
        spikes.index = pd.MultiIndex.from_frame(idx)
        spike_waveforms = np.load(exp.ephys_params[probe]['waveforms'])
        width = {}
        for u in cluster_metrics.index:
            wav = spike_waveforms[u, cluster_metrics.loc[u, 'peak_channel'], :]
            wav_duration = np.abs(np.argmin(wav)-np.argmax(wav))
            width[u] = wav_duration / exp.ephys_params[probe]['ap_sample_rate']
        cluster_metrics = cluster_metrics.join(pd.Series(width, name='waveform_width'))

        with open(exp.ephys_params[probe]['probe_info'], 'r') as f:
            areas = json.load(f)['area_ch']
        areas = pd.Series(areas, name='area')
        layers = areas.str.extract('(\d.*)')[0].fillna('').rename('layer')
        areas = areas.str.rstrip('12/3456ab').fillna('')
        cluster_metrics['area'] = cluster_metrics.peak_channel.map(lambda x: areas.loc[x])
        cluster_metrics['layer'] = cluster_metrics.peak_channel.map(lambda x: layers.loc[x])
        cluster_metrics['FS_RS'] = cluster_metrics.waveform_width.map(lambda x: 'FS' if x<0.0004 else 'RS')
        spikes.columns = pd.MultiIndex.from_frame(cluster_metrics[['area', 'layer', 'FS_RS', 'peak_channel']].reset_index().rename({'cluster_id':'unit'}, axis=1))
        
        # remove stimulation artifact
        spikes.loc[-0.001, 0.002] = False
        
        spikes.to_pickle(fname)
    return spikes

def get_mean_firing(rec_folder, probe, repeat_spikes=False):
    exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
    fname = f'../tiny-blue-dot/zap-n-zip/sg/spikes_stim_aligned/fr_{exp.mouse}_{path.basename(path.dirname(exp.experiment_folder))}_{probe}.pkl'
    if path.exists(fname):
        return pd.DataFrame(pd.read_pickle(fname), dtype='int16')
    spikes = load_spikes(rec_folder, probe, repeat=repeat_spikes)
    fr = pd.DataFrame((spikes.groupby(['stim_type', 'parameter', 'sweep', 'time']).mean()*1000).round(), dtype='int16')
    fr.to_pickle(fname)
    return pd.DataFrame(fr, dtype='int16')

In [11]:
def smoothen_df(df, win=10):
    win = sp.signal.hann(win)
    df2 = df.copy()
    for t in df.columns:
        df2[t] = sp.signal.convolve(df[t], win, mode='same') / sum(win)
    return df2

def compute_csd(lfp_df, spacing, method='sl', sf=2500, smoothen=False, **kwargs):
    if method=='sl':
        # Need to pad lfp channels for Laplacian approx.
        padded_lfp = np.pad(
            lfp_df, pad_width=((1, 1), (0, 0)), mode='edge'
        )
        csd = (1 / (spacing ** 2)) * (
            padded_lfp[2:, :] - (2 * padded_lfp[1:-1, :]) + padded_lfp[:-2, :]
        )
    else:
        params = dict(diam=200E-6*pq.m, sigma=0.3*pq.S/pq.m, sigma_top=0.3*pq.S/pq.m, f_type='gaussian', f_order=(3, 1))
        if method=='StepiCSD':
            params.update(dict(h=20E-6*pq.m, tol=1E-12))
        if method=='SplineiCSD':
            params.update(dict(num_steps=len(lfp_df), tol=1E-12, f_order=(20, 5)))
        params.update(kwargs)
        coord = lfp_df.columns.remove_unused_levels().to_frame()[['depth']].values*pq.um
        # select only the vertical dimension
        coord = coord[:, np.newaxis]
        neo_lfp = AnalogSignal(lfp_df, units='uV', sampling_rate=sf*pq.Hz)
        csd = elephant.current_source_density.estimate_csd(neo_lfp, coords=coord, method=method, **params)
    csd = pd.DataFrame(csd, index=lfp_df.index, columns=lfp_df.columns)
    if smoothen:
        csd = smoothen_df(csd.T).T
    return csd

In [12]:
def get_csd(rec_folder, probe, repeat=False, repeat_lfp=False, smoothen_lfp=False, reref_saline=True, separate_running=False):
    print(rec_folder)
    exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
    fname = f'../tiny-blue-dot/zap-n-zip/sg/csd/{exp.mouse}_{path.basename(path.dirname(exp.experiment_folder))}_{probe}{"_running_separated" if separate_running else ""}.pkl'
    if not reref_saline:
        fname += '.noreref'
    if path.exists(fname) and not repeat:
        csd = pd.read_pickle(fname)
    else:
        print('Computing CSD.')
        lfp = get_mean_lfp(rec_folder, probe, repeat_lfp, separate_running=separate_running)
        
        if reref_saline:
            # rereference to saline channels
            with open(exp.ephys_params[probe]['probe_info'], 'r') as f:
                probe_info = json.load(f)
            surface_channel = int(probe_info['surface_channel'])
            air_channel = int(probe_info['air_channel'])
            top_saline_channels = np.arange(air_channel-1, surface_channel, -1)[:6]
            lfp = (lfp.T-lfp[top_saline_channels].mean(1)).T
            
            areas = probe_info['area_ch']
            areas = pd.Series(areas, name='area')
            layers = areas.str.extract('(\d.*)')[0].fillna('').rename('layer')
            areas = areas.str.rstrip('12/3456ab').fillna('')
            lfp.columns = pd.MultiIndex.from_frame(lfp.columns.to_frame(index=False).join(areas))
            lfp = lfp.droplevel('area', axis=1)
        
        lfp = lfp.loc[:, ::2]
        if smoothen_lfp:
            print('smoothening...', end='')
            lfp = smoothen_df(lfp.T).T
            print('done.')
        try:
            csd = compute_csd(lfp, spacing=20, method='DeltaiCSD', smoothen=True)
        except:
#             print('Failed to compute CSD. Returning LFP for further debugging.')
            return None#lfp
        
        with open(exp.ephys_params[probe]['probe_info'], 'r') as f:
            areas = json.load(f)['area_ch']
        areas = pd.Series(areas, name='area')
        layers = areas.str.extract('(\d.*)')[0].fillna('').rename('layer')
        areas = areas.str.rstrip('12/3456ab').fillna('')
        csd.columns = pd.MultiIndex.from_frame(csd.columns.to_frame().droplevel('depth').join(areas).join(layers))
        csd.to_pickle(fname)
    return csd

In [13]:
# plot along with layer and area information
def plot_csd_panel(df, ax, demean=False, normalize=False, thresh=0.999, vx=None):
    try:
        df = df.loc[df.name]
    except:
        pass
    _df = df.mean(1).abs()
#     vx = df[_df < _df.quantile(0.999)].unstack(level=list(range(df.index.nlevels))).abs().quantile(0.999)
#     Vx = vx
#     if normalize:
# #         print('normalizing')
#         df = df / vx
#         vx = 1
# #     print(vx)
    if demean:
        df = df-df.loc[:0].mean()
    if vx is None:
        vx = np.quantile(df.loc[0.005:].abs().values, thresh)
    Vx = vx
    im = ax.imshow(df.T, aspect='auto', extent=[
        df.index.get_level_values('time')[0],
        df.index.get_level_values('time')[-1],
#         0, len(df.columns)
        df.columns.get_level_values('depth')[0],
        df.columns.get_level_values('depth')[-1],
    ], interpolation='none', cmap='jet', vmin=-vx, vmax=vx, origin='lower')
    layers = df.T.groupby(['area', 'layer'], sort=False).size()[::-1].cumsum()
    
    for a, y in (layers*20).items():
        if y - 20*layers.shift(1).loc[a] > 80 or np.isnan(layers.shift(1).loc[a]):
            ax.axhline(df.columns.get_level_values('depth')[-1]-y, c='k', alpha=0.4)
            ax.annotate(
                a[0]+a[1], (0.99, df.columns.get_level_values('depth')[-1]-y),
                xycoords=('axes fraction', 'data'), fontsize=7,
                va='bottom', ha='right', annotation_clip=True
            )
    return im, Vx

def plot_area_bar(csd, aax):
    aax.tick_params(labelleft=False)
    # plot area reference
    _alois = csd.columns.to_frame().apply(lambda r: f'{r.area}{r.layer}', 1)
    ids, vals = np.unique(_alois, return_inverse=True)
#     ids, vals = np.unique(csd.columns.to_frame().area, return_inverse=True)
    aax.imshow(
        vals[:, np.newaxis], aspect='auto', origin='lower',
        extent=[
            0, 1, csd.columns.get_level_values('depth')[0],
            csd.columns.get_level_values('depth')[-1]
        ], cmap=mpl.colors.ListedColormap([aloi_colors.get(x, cm.Greys(0.5, 0.5)) for x in ids])#cm.Accent
    )
    aax.set_xlabel('area')
    for i, y in enumerate(np.r_[np.arange(len(vals)-1)[np.diff(vals)!=0], len(vals)-1]):
        aax.annotate(ids[vals[y]], (0, csd.columns.get_level_values('depth')[y]), va='top', fontsize=7)
    return

def plot_csd_firing_panel(csd_df, fr_df=None, ax=None, vx=None, aois_only=True):
    if aois_only:
        csd_df = csd_df.swaplevel(0, 'area', axis=1)[
            [x for x in aois if x in csd_df.columns.get_level_values('area').unique()]
        ].sort_index(level='channel', axis=1)
        if fr_df is not None:
            fr_df = fr_df.swaplevel(0, 'area', axis=1)[
                [x for x in aois if x in fr_df.columns.get_level_values('area').unique()]
            ].sort_index(level='peak_channel', axis=1)
    else:
        csd_df = csd_df.swaplevel(0, 'area', axis=1).sort_index(level='channel', axis=1)
        if fr_df is not None:
            fr_df = fr_df.swaplevel(0, 'area', axis=1).sort_index(level='peak_channel', axis=1)
    
    if vx is None:
        vx = 0.5
    
    if ax is None:
        f, ax = plt.subplots(1, 1, figsize=(4, 3), tight_layout=True)

    ax.axvline(0, c='k', alpha=0.4)
    im = ax.imshow(
        csd_df.T, aspect='auto', cmap='jet', vmin=-vx, vmax=vx, origin='lower',
        extent=[
            csd_df.index.get_level_values('time')[0],
            csd_df.index.get_level_values('time')[-1],
            0, len(csd_df.columns)
        ]
    )

    als = csd_df.groupby(level=['area', 'layer'], sort=False, axis=1).size()
    for k, v in als.cumsum().shift(fill_value=0).items():
        ax.axhline(v, c='k', alpha=0.3)
        ax.annotate(
            f'{k[0]}{k[1]}', (0, v), xycoords=('axes fraction', 'data'),
            fontsize=7, va='bottom', ha='left', annotation_clip=True
        )
    
    if fr_df is not None:
        mfr = fr_df.T.groupby(['area', 'layer', 'FS_RS'], sort=False).mean().unstack().swaplevel(
            0, 'FS_RS', axis=1).sort_index(axis=1)
        if 'stim_type' in mfr.columns.names:
            mfr = mfr.droplevel('stim_type', axis=1)
        mfr_zs = mfr.groupby(level='FS_RS', axis=1).apply(lambda df: (
            (df.swaplevel(axis=1).T-df.swaplevel(axis=1).T.loc[:0].mean())/\
            df.swaplevel(axis=1).T.loc[:0].std()
        ).T.swaplevel(axis=1).rolling(10, center=True, axis=1).mean())

    #     display(als)
    #     display(mfr_zs.index.get_level_values('area').unique())
    #     display(csd_df.columns.get_level_values('area').unique())
        mfr_plotready = mfr_zs.apply(
            lambda r: als.cumsum().shift(fill_value=0).loc[r.name]+1+\
            r*als.loc[r.name]/max(
                r.loc[(slice(None), slice(0.05, 5))].max()-r.loc[(slice(None), slice(0.05, 5))].min(),
                10), axis=1
        )

        if 'FS' in mfr_plotready.columns.get_level_values(0).unique():
            mfr_plotready['FS'].T.plot(ax=ax, legend=False, c=cm.Greens(0.9, 0.99), lw=2)
        if 'RS' in mfr_plotready.columns.get_level_values(0).unique():
            mfr_plotready['RS'].T.plot(ax=ax, legend=False, c=cm.Purples(0.7, 0.99), lw=2)

    ax.set_ylim(0, len(csd_df.columns))
    ax.set_yticks([])
    return ax

## Single experiment tests

In [53]:
mouse = 'mouse569062'
expt = good_expt[good_expt.mouse_name==mouse].iloc[0].exp_name
probe = f"probe{good_expt[good_expt.mouse_name==mouse].iloc[0].Npx.split(',')[2]}"
rec_folder = f'../tiny-blue-dot/zap-n-zip/EEG_exp/{mouse}/{expt}/experiment1/recording1/'

In [54]:
# # test / debug LFP computation pipeline (loaf_lfp function)

# rec_folder = f'../tiny-blue-dot/zap-n-zip/EEG_exp/{mouse}/{expt}/experiment1/recording1/'
# exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
# samp_rate = exp.ephys_params[probe]['lfp_sample_rate']
# stim_log = pd.read_csv(exp.stimulus_log_file)
# stim_log.rename_axis(index='stim_id', inplace=True)

# lfp = np.memmap(exp.ephys_params[probe]['lfp_continuous'], dtype='int16', mode='r')
# lfp = np.reshape(lfp, (int(lfp.size/exp.ephys_params[probe]['num_chs']), exp.ephys_params[probe]['num_chs']))

# timestamps = np.load(exp.ephys_params[probe]['lfp_timestamps'])

# lfp = pd.DataFrame(lfp, index=timestamps)

# print('Timestamps shape:', timestamps.shape)
# print('LFP shape:', lfp.shape)

# idx = stim_log.reset_index().set_index('onset')
# # display(idx.head())
# idx.index = idx.index - 1
# idx = idx.reindex(timestamps, method='ffill').reset_index()
# # display(idx)

# def _reset_index_time(df):
#     df['onset'] = (df.onset - df.onset.iloc[0] - 1).round(4)
#     return df
# idx = idx.groupby('stim_id', dropna=False).apply(_reset_index_time).drop(['offset', 'duration'], axis=1).rename(columns={'onset':'time'})
# display(idx)

# lfp = lfp.loc[timestamps[idx.index]]

# # reref with top saline channels
# with open(exp.ephys_params[probe]['probe_info'], 'r') as f:
#     probe_info = json.load(f)
# surface_channel = int(probe_info['surface_channel'])
# air_channel = int(probe_info['air_channel'])
# top_saline_channels = np.arange(air_channel-1, surface_channel, -1)[:6]

# top_saline_channels

In [55]:
exp = EEGexp(rec_folder, preprocess=False, make_stim_csv=False)
stim_log = pd.read_csv(exp.stimulus_log_file)
stim_log.rename_axis(index='stim_id', inplace=True)

csd = get_csd(rec_folder, probe, repeat_lfp=False, repeat=True, separate_running=True)
# spikes = load_spikes(rec_folder, probe, repeat=True)
fr = get_mean_firing(rec_folder, probe, repeat_spikes=False)

aparams = list(stim_log.apply(lambda row: (row.stim_type, row.parameter), axis=1).unique())
print('Available parameters:')
print(aparams)

Experiment type: electrical and sensory stimulation
../tiny-blue-dot/zap-n-zip/EEG_exp/mouse569062/estim_vis_2021-02-18_11-17-51/experiment1/recording1/
Experiment type: electrical and sensory stimulation
Computing CSD.
Experiment type: electrical and sensory stimulation


/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/allensdk/brain_observatory/sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'eyetracking'}. Consider updating the sync file line labels.
  self._check_line_labels()


parameter  sweep
15         0        120
           1        120
           2        120
35         0        120
           1        120
           2        120
60         0        120
           1        120
           2        120
white      0        120
           1        120
           2        120
dtype: int64

parameter  sweep
15         0         90
           1        119
           2        116
           10        30
           11         1
           12         4
35         0         97
           1        118
           2        116
           10        23
           11         2
           12         4
60         0         84
           1        119
           2        115
           10        36
           11         1
           12         5
white      0        120
           1        109
           2        120
           11        11
dtype: int64

Experiment type: electrical and sensory stimulation
separating running sweeps


/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/allensdk/brain_observatory/sync_dataset.py:109: UserWarning: The loaded sync file contains the following deprecated line label keys: {'eyetracking'}. Consider updating the sync file line labels.
  self._check_line_labels()


parameter  sweep
15         0        120
           1        120
           2        120
35         0        120
           1        120
           2        120
60         0        120
           1        120
           2        120
white      0        120
           1        120
           2        120
dtype: int64

parameter  sweep
15         0         90
           1        119
           2        116
           10        30
           11         1
           12         4
35         0         97
           1        118
           2        116
           10        23
           11         2
           12         4
60         0         84
           1        119
           2        115
           10        36
           11         1
           12         5
white      0        120
           1        109
           2        120
           11        11
dtype: int64

discrete filter coefficients: 
b = [ 0.607 1.000 0.607 ],                
a = [ 2.213 ]
Experiment type: electrical and sensory stimulation
Available parameters:
[('biphasic', '35'), ('biphasic', '15'), ('biphasic', '60'), ('circle', 'white')]


In [56]:
param = '35'
csd_df = csd.xs(param, level='parameter').xs(10, level='sweep')
fr_df = fr.xs(param, level='parameter').xs(0, level='sweep')
ax = plot_csd_firing_panel(csd_df, aois_only=False, vx=1.5)
ax.set_xlim(-0.1, 0.5)
ax.set_title(f'running trials ({probe}, {param} $\mu$A)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
param = '35'
csd_df = csd.xs(param, level='parameter').xs(0, level='sweep')
fr_df = fr.xs(param, level='parameter').xs(0, level='sweep')
ax = plot_csd_firing_panel(csd_df, aois_only=False, vx=1.5)
ax.set_xlim(-0.1, 0.5)
ax.set_title(f'resting trials ({probe}, {param} $\mu$A)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
# # compare with uncorrected CSD
# fname = f'../tiny-blue-dot/zap-n-zip/sg/csd/{exp.mouse}_{path.basename(path.dirname(exp.experiment_folder))}_{probe}_uncorrected.pkl'
# csd_uncorrected = pd.read_pickle(fname)

# csd_dfu = csd_uncorrected.xs('50', level='parameter').xs(0, level='sweep')

# f, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6), tight_layout=True, sharex=True, sharey=True)
# plot_csd_firing_panel(csd_df, aois_only=False, vx=1.5, ax=ax2)
# plot_csd_firing_panel(csd_dfu, aois_only=False, vx=1.5, ax=ax1)
# ax1.set_xlim(-0.1, 0.5)
# ax1.set_title(f'{mouse} {probe}')
# ax1.set_ylabel('uncorrected')
# ax2.set_ylabel('corrected for sampling time\nand rereferencing artifacts')
# ax2.set_xlabel('time (s)');

In [17]:
ncol = len(csd.index.levels[1])
nrow = len(csd.index.levels[2])
f, axes1 = plt.subplots(
    nrow, ncol, figsize=(3*ncol, 2.4*nrow),
    constrained_layout=True, sharex=True, sharey=True
)

axes = {
    p : axes1.T.flatten()[i] for i, p in enumerate(sorted(csd.index.droplevel(['time', 'stim_type']).unique()))
}

for n, ax in axes.items():
    ax.set_title(f'stim: {n[0]}; state: {n[1]:.0f}')
    ax.set_xlim(-0.24, 0.48)

for ax in axes1.flatten()[len(axes):]:
    ax.set_visible(False)
csd.swaplevel(0, -1).groupby(['parameter', 'sweep']).apply(lambda df: plot_csd_panel(df, ax=axes[df.name], demean=True))

f.suptitle(f'mouse {exp.mouse}, {probe}');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
# CSD artifact in saline
# lfp = get_mean_lfp(rec_folder, probe)
# _lfp = lfp.xs('50', level='parameter').xs(0, level='sweep').loc['biphasic']

# csd = get_csd(rec_folder, probe, reref_saline=False)
# _csd = csd.xs('50', level='parameter').xs(0, level='sweep').loc['biphasic']

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 7), tight_layout=True, sharex=True)
_lfp[[214, 210, 206, 202, 198, 194, 190, 186]].plot(ax=ax1, c='g', alpha=0.5)
_lfp[[380, 376, 372, 368, 364, 360, 356, 352]].plot(ax=ax1, c='b', alpha=0.5)
# _lfp[[1, 5, 9, 13, 17, 21, 25, 29]].plot(ax=ax1, c='r', alpha=0.5)
ax1.legend(ncol=2, fontsize=8, loc=(1.01, 0))
ax1.set_ylabel('lfp')

ax2.imshow(_lfp.T, aspect='auto', extent=[_csd.index[0], _csd.index[-1], 0, len(_csd.columns)], cmap='jet', origin='lower')#, vmax=1.5, vmin=-1.5)
ax2.axhline(382/2, c='k')
ax2.axhline(354/2, c='k')
ax2.axhline(212/2, c='k')
ax2.axhline(184/2, c='k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:307: MatplotlibDeprecationWarning: 
The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get_visible()
/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:307: MatplotlibDeprecationWarning: 
The colNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().colspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get_visible()
/home/saurabh.gandhi/Projects/.env_zz/lib/python3.7/site-packages/pandas/plotting/_matplotlib/tools.py:307: MatplotlibDeprecationWarning: 
The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get

In [52]:
# numbers for Atle
_lfp = _lfp.reindex(csd.columns, axis=1)

_sd = _lfp.loc[0.03:0.08].groupby(level='area', axis=1).apply(lambda df: df.values.std())
display((_sd / _sd['null']).sort_values())
_sd_non_null = _lfp[_lfp.columns[_lfp.columns.get_level_values('area')!='null']].values.std()
print(_sd_non_null / _sd['null'])

area
ZI              0.167236
HY              0.216076
MGm             0.259412
MGv             0.325341
TH              0.388917
MGd             0.448040
LGd-co          0.656587
DG-po           0.669767
null            1.000000
LGd-sh          1.000805
DG-sg           1.134197
fiber tracts    1.160408
bsc             1.184530
VISp            1.386037
or              1.391447
fp              1.509964
alv             1.597870
CA              1.725808
DG-mo           1.906511
dtype: float64

0.8313461357282858


## Compute / load CSD for all available experiments

In [ ]:
# load all CSDs
csds = {}
areas = {}
for mouse in good_expt.mouse_name.unique()[::-1]:
    for expt in good_expt[good_expt.mouse_name==mouse].exp_name:
        for probe in tqdm(good_expt[(good_expt.mouse_name==mouse)&(good_expt.exp_name==expt)].Npx.values[0].split(','), desc=f'{mouse} {expt}'):
            try:
                probe = f'probe{probe}'
                rec_folder = f'../tiny-blue-dot/zap-n-zip/EEG_exp/{mouse}/{expt}/experiment1/recording1/'
                with capture_output():
                    csd = get_csd(rec_folder, probe, repeat=True, repeat_lfp=False, separate_running=True)
                    areas[(mouse, expt, probe)] = csd.columns.to_frame(index=False)
                    csds[(mouse, expt, probe)] = csd.droplevel(['area', 'layer'], axis=1)
            except Exception as e:
                print(f'Failed {mouse} {expt} {probe}: {e}')
csds = pd.concat(csds, names=['mouse', 'expt', 'probe', 'stim_type', 'parameter', 'sweep', 'time']).droplevel('stim_type').sort_index()
areas = pd.concat(areas).sort_index()

In [ ]:
# load all firing rates
frs = {}
fr_areas = {}
for mouse in good_expt.mouse_name.unique():
    for expt in good_expt[good_expt.mouse_name==mouse].exp_name:
        for probe in tqdm(good_expt[(good_expt.mouse_name==mouse)&(good_expt.exp_name==expt)].Npx.values[0].split(','), desc=f'{mouse} {expt}'):
            try:
                probe = f'probe{probe}'
                rec_folder = f'../tiny-blue-dot/zap-n-zip/EEG_exp/{mouse}/{expt}/experiment1/recording1/'
                with capture_output():
                    frs[(mouse, expt, probe)] = get_mean_firing(rec_folder, probe, repeat_spikes=False)
                    fr_areas[(mouse, expt, probe)] = frs[(mouse, expt, probe)].columns.to_frame(index=False)
                    frs[(mouse, expt, probe)].columns = range(len(frs[(mouse, expt, probe)].columns))
            except Exception as e:
                print(f'Failed {mouse} {expt} {probe}: {e}')
frs = pd.concat(frs, names=['mouse', 'expt', 'probe', 'stim_type', 'parameter', 'sweep', 'time']).droplevel('stim_type').sort_index()
fr_areas = pd.concat(fr_areas).sort_index()

# Plot CSDs and compare timings across probes

In [ ]:
def plot_all_cd_responses(mouse, expt, param, include=None, thresh=0.999, vx=None, xmin=-0.1, xmax=0.8, aois_only=True, figsize=None):
    try:
        csd_df = csds.loc[(mouse, expt)].xs(param, level='parameter').sort_index()
        # rename sweeps according to state
        csd_df = csd_df.rename(
            index={i:f'{i:.0f}: {x}' for i, x in enumerate(
                good_expt[(good_expt.mouse_name==mouse)&\
                          (good_expt.exp_name==expt)].iloc[0]['brain states'].split('/')
            )}, level='sweep'
        )
    except:
        raise ValueError(
            'Available param values are: ',
            csds.loc[(mouse, expt)].index.get_level_values('parameter').unique()
        )

    probes = csd_df.index.remove_unused_levels().levels[0]
    sweeps = csd_df.index.remove_unused_levels().levels[-2]
    if include is not None:
        sweeps = sweeps[np.array(include)]
    if figsize is None:
        figsize=(3*len(sweeps), 2.3*len(probes))
    f, axes1 = plt.subplots(
        len(probes), len(sweeps), tight_layout=True,
        figsize=figsize, sharex=True, sharey=True,
        gridspec_kw=dict(hspace=0), squeeze=False
    )
    axes = {x:axes1.flatten()[i] for i, x in enumerate(itertools.product(probes, sweeps))}

    def _plot_csd_panel(df):
        name = df.name
        if name[1] not in sweeps:
            return
        df.columns = pd.MultiIndex.from_frame(areas.loc[(mouse, expt, df.name[0])])
        if aois_only:
            df[df.columns[~df.columns.get_level_values('area').isin(aois)]] = 0
        return plot_csd_panel(df.droplevel(['sweep', 'probe']), ax=axes[name], thresh=thresh, demean=True, vx=vx)

    im_vx = csd_df.groupby(['probe', 'sweep'], sort=False).apply(_plot_csd_panel)

    for n, ax in axes.items():
        if ax in axes1.T[0]:
            ax.set_ylabel(n[0])
            ax.set_yticks([])
        if ax in axes1[0]:
            ax.set_title(f'{n[1]}')
        if ax in axes1[-1]:
            ax.set_xlabel('time (s)')

    for ax in axes1.flatten():
        ax.set_xlim(xmin, xmax)
    
    return f, axes, axes1, im_vx

def plot_all_cd_fr_responses(mouse, expt, param, include=None, vx=None, xmin=-0.1, xmax=0.8, aois_only=True, figsize=None):
#     try:
    csd_df = csds.loc[(mouse, expt)].xs(param, level='parameter').sort_index()
    fr_df = frs.loc[(mouse, expt)].xs(param, level='parameter').sort_index()
#     fr_df = fr_df
    # rename sweeps according to state
    csd_df = csd_df.rename(
        index={i:f'{i:.0f}: {x}' for i, x in enumerate(
            good_expt[(good_expt.mouse_name==mouse)&\
                      (good_expt.exp_name==expt)].iloc[0]['brain states'].split('/')
        )}, level='sweep'
    )
    fr_df = fr_df.rename(
        index={i:f'{i:.0f}: {x}' for i, x in enumerate(
            good_expt[(good_expt.mouse_name==mouse)&\
                      (good_expt.exp_name==expt)].iloc[0]['brain states'].split('/')
        )}, level='sweep'
    )
#     except:
#         raise ValueError('Available param values are: ', csds.loc[(mouse, expt)].index.get_level_values('parameter').unique())

    probes = csd_df.index.remove_unused_levels().levels[0]
    sweeps = csd_df.index.remove_unused_levels().levels[-2]
    if include is not None:
        sweeps = sweeps[np.array(include)]
    if figsize is None:
        figsize=(3*len(sweeps), 2.3*len(probes))
    f, axes1 = plt.subplots(
        len(probes), len(sweeps), tight_layout=True,
        figsize=figsize, sharex=True, sharey=True,
        gridspec_kw=dict(hspace=0), squeeze=False
    )
    axes = {x:axes1.flatten()[i] for i, x in enumerate(itertools.product(probes, sweeps))}
    
#     display(csd_df)
    for p, s in csd_df.index.droplevel('time').unique():
        if s not in sweeps:
            continue
        df = csd_df.loc[(p, s)]
        df.columns = pd.MultiIndex.from_frame(areas.loc[(mouse, expt, p)])
        
        _df = fr_df.loc[(p, s)].dropna(axis=1, how='all')
        _df.columns = pd.MultiIndex.from_frame(fr_areas.loc[(mouse, expt, p)])
        
        plot_csd_firing_panel(df, _df.dropna(axis=1, how='all'), axes[(p, s)], vx=vx, aois_only=aois_only)

    for n, ax in axes.items():
        if ax in axes1.T[0]:
            ax.set_ylabel(n[0])
            ax.set_yticks([])
        if ax in axes1[0]:
            ax.set_title(f'{n[1]}')
        if ax in axes1[-1]:
            ax.set_xlabel('time (s)')

    for ax in axes1.flatten():
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(0, 192)
    
    return f, axes, axes1

In [ ]:
params = csds.loc[(mouse, expt)].index.remove_unused_levels().levels[1]
f'{(np.median([int(p) for p in params if len(p)<4])):.0f}'

In [46]:
[int(p) for p in params if len(p)<4]

[100, 60, 80]

In [49]:
for mouse in csds.index.levels[0]:
    for expt in good_expt[good_expt.mouse_name==mouse].exp_name:
        try:
            stim_area = good_expt.set_index(
                ['mouse_name', 'exp_name']
            ).loc[(mouse, expt), 'CCF area stim electrode (surface,tip)'].split(',')[1]
        except:
            stim_area = 'unknown'
        params = csds.loc[(mouse, expt)].index.remove_unused_levels().levels[1]
        try:
            param = f'{(np.median([int(p) for p in params if len(p)<4])):.0f}'
        except:
            param = np.median([int(p) for p in params if len(str(p))<4])
        vx = 1.5
        try:
            # f, axes, axes1, im_vx = plot_all_cd_responses(mouse, param, include=[0, 1], thresh=0.99, vx=vx, xmin=-0.1, xmax=0.5, aois_only=False, figsize=(14, 8))
            f, axes, axes1 = plot_all_cd_fr_responses(mouse, expt, param, include=[0, 1], vx=vx, xmin=-0.1, xmax=0.5, aois_only=False, figsize=(12, 6))
            f.suptitle(f'{mouse} ({stim_area}, {param}) (CSD range: {vx}$\mu$A/mm$^2$)');
        except:
            print(f'Failed for {mouse} {param}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …